# 2022 NYC Open Data Week
# Civic Analytics using Open Data

https://www.urbanintelligencelab.org \ https://github.com/UrbanIntelligenceLab \ @UrbanIntelLab

In [ ]:
import pandas as pd
import numpy as np
from urllib.parse import urlencode
import datetime
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Access and Query 311 Data using Socrata Open API 1
## Goal 1: Access 311 data for multiple cities without downloading
### 1) Access NYC 311 data using Socrata API
* You can find Socrata API endpoint on the Open Data webpage.
![screenshot](../image/nyc_open_data_screenshot.png)

In [ ]:
# Define API endpoint
API = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv?'

# Get a data header
query = {'$select': '*',
        }
df = pd.read_csv(API+urlencode(query), parse_dates=['created_date'])
print (len(df))
df.head()

### 2) Show header with selected columns

In [ ]:
# 'created_date', 'agency', 'complaint_type', 'location_type', 'incident_zip', 'latitude', 'longitude'

# Get a data header
query = {'$select': 'created_date, agency, complaint_type, location_type, incident_zip, latitude, longitude',
        }
df = pd.read_csv(API+urlencode(query), parse_dates=['created_date'])
df.head()

### 3) Count complaint volume
* Total number of 311 complaints since 2010
* Yearly counts of 311 complaints
* Montly counts of 311 complaints in 2021

In [ ]:
# Total volume since 2010
query = {'$select': 'count(*)',
         }
df = pd.read_csv(API+urlencode(query))
df.head()

In [ ]:
# Yearly counts
query = {'$select': 'date_trunc_y(created_date) as year, count(*) as yearly_count',
         '$group': 'year',
        }
df = pd.read_csv(API+urlencode(query), parse_dates=['year'])
df.head()

In [ ]:
# Monthly counts of 311 complaints in 2021
query = {'$select': 'date_trunc_ym(created_date) as month, count(*) as monthly_count',
         '$where': 'created_date between "2021-01-01" and "2021-12-31"',
         '$group': 'month',
        }
df = pd.read_csv(API+urlencode(query), parse_dates=['month'])
df.sort_values(by='month').head()

## Goal 2: Count 311 service requests for multiple cities
### 4) Apply to multi-city 311 data to retrieve a total number of 311 complaints since data has been collected

In [ ]:
# Socrata API endpoint for each city should be collected first. Here, we use a pre-collected table for 6 cities.
# New York, Chicago, Montgomery, San Francisco, Austin, and Baltimore

city_list = pd.read_csv('../data/example_cities_socrata.csv')
city_list[['city_state', 'Year','api_url', 'unique_column', 'date_column']].head()

### 5) Create a summary table with total counts of complaints (overlapping period)

In [ ]:
# Use for loops to count 311 complaint volumes for each city

cities = []
com_vol_ovpr = []

for i in range(len(city_list)):
    try:  
        query = {'$select': 'count(*)',
                 '$where': '%s between "2021-01-01" and "2021-12-31"'%(city_list['date_column'].iloc[i]),
                }
        df = pd.read_csv(city_list['api_url'].iloc[i]+'?'+urlencode(query))
        com_vol_ovpr.append(df.values[0][0])
        cities.append(city_list['city'].iloc[i])
    
    except:
        pass
print ("Processing completed")

result = pd.DataFrame()
result['city'] = cities
result['com_vol_ovpr'] = com_vol_ovpr
result['com_vol_ovpr'] = result['com_vol_ovpr'].astype(float)

result

## Goal 3: Create a plot to compare values across different cities
### 6) Create a bar chart (descending)

In [ ]:
result.index = result['city']

fig = plt.figure(figsize=(4,3)) # Create matplotlib figure
ax = fig.add_subplot(111) # Create matplotlib axes
width = 0.4
result.sort_values(by='com_vol_ovpr', ascending=False).com_vol_ovpr.plot(kind='bar', color='SteelBlue', ax=ax, width=width)
ax.set_ylabel('311 Complaint Volume (total)')

![bar_chart_multicity](../image/bar_chart_multicity.png)

# Access and Query 311 Data using Socrata Open API 2
## Goal 1: Access NYC 311 data and Count 2019 & 2020 citizen service requests
### 1) Access NYC 311 and compare 2019 & 2020 (pre-pandemic vs. post-pandemic) complatint volume

In [ ]:
# Total complaints March 1, 2019 ~ June 30, 2019
query_2019 = {'$select': 'count(*) as total_count_2019',
         '$where': 'created_date between "2019-03-01" and "2019-06-30"',
        }
df_2019 = pd.read_csv(API+urlencode(query_2019))

# Total complaints March 1, 2020 ~ June 30, 2020 (Pandemic period)
query_2020 = {'$select': 'count(*) as total_count_2020',
         '$where': 'created_date between "2020-03-01" and "2020-06-30"',
        }
df_2020 = pd.read_csv(API+urlencode(query_2020))

df = pd.concat([df_2019, df_2020], axis=1)
df

## Goal 2: Create plots to understand most reported complaint types
### 2) Create a plot - top 50 complaint types (total)

In [ ]:
DATE_CONDITION = 'created_date between "2020-02-01" and "2020-08-10"'

# get complaint volume
query = {'$select': 'complaint_type, date_trunc_ymd(created_date) as date, count(*)',
         '$where': DATE_CONDITION,
         '$group': 'complaint_type, date',
         '$limit': 1000*1000,
        }
df = pd.read_csv(API+urlencode(query), parse_dates=['date'])
df.head()

In [ ]:
df = df.pivot_table(index='date', columns='complaint_type', values='count', fill_value=0)
df.head()

In [ ]:
truncated = df.sum(axis=0).sort_values(ascending=False).head(50)
truncated[:5]

In [ ]:
temp = df[truncated.index].copy().reset_index(drop=True)
ax = temp.plot.area(stacked=True, figsize=(16,10), 
                    title='Volume of Top 50 Complaint Types (Feb 01 - Aug 10)',
                    cmap=ListedColormap(list(cm.get_cmap('tab20').colors)*3))
# legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], 
          title='Top 50 Complaint Types\n(Ascending order of volume)', 
          fontsize=9,
          bbox_to_anchor=(1.03,1))

# date labels
x_indices = range(0, len(temp.index), 7)
date_index = df[truncated.index].index
x_labels = date_index.strftime('%h %d (%a)').tolist()
ax.set_xticks(x_indices)
_ = ax.set_xticklabels([x_labels[i] for i in x_indices], rotation=90)


### annotation
# annotate covid dates
dates = {
    '2020-03-20': 'Shelter In Place',
    '2020-06-08': 'Phase 1',
    '2020-06-22': 'Phase 2',
    '2020-07-06': 'Phase 3',
    '2020-07-20': 'Phase 4',
}
annotation = [(i, date) for i, date in enumerate(date_index.strftime('%Y-%m-%d').tolist()) 
     if date in dates]
for annot_i, annot_text in annotation: 
    ax.axvline(annot_i, color='gold', linestyle='--')
    ax.text(x=annot_i+.5, y=21000, s='%s\n%s'%(dates[annot_text], 
                                               date_index[annot_i].strftime('%h %d')))
    
# annotate storm dates
dates = ['2020-07-10', '2020-08-04']
date_labels = ['Storm Fay', 'Storm Isais']
annotation = [i for i, date in enumerate(date_index.strftime('%Y-%m-%d').tolist()) if date in dates]
for i, date_i in enumerate(annotation): 
    ax.axvline(date_i, color='grey', linestyle='--')
    ax.text(x=date_i+.5, y=20000, s='%s\n%s'%(date_labels[i], 
                                               date_index[date_i].strftime('%h %d')))

# annotate heat advisory dates
dates = ['2020-07-19', '2020-07-23', '2020-07-26', '2020-08-10']
annotation = [i for i, date in enumerate(date_index.strftime('%Y-%m-%d').tolist()) if date in dates]
for i, date_i in enumerate(annotation): 
    ax.axvline(date_i, color='salmon', linestyle='--')

### 3) Create a plot - top 50 complaint types (stacked proportional)

In [ ]:
# Calauclate proportional values for each complaint type
temp = df[truncated.index].copy()
temp = temp.div(df[truncated.index].sum(axis=1), axis=0)
temp.head()

In [ ]:
ax = temp.plot.area(stacked=True, figsize=(16,10), 
                    title='% Daily Volume of Top 50 Complaint Types (Feb 01 - Aug 10)',
                    cmap=ListedColormap(list(cm.get_cmap('tab20').colors)*3)
                   )
ax.set_xlabel('')
# legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], 
          title='Top 50 Complaint Types\n(Ascending order of volume)', 
          fontsize=9,
          bbox_to_anchor=(1,1))

# date labels
x_indices = range(0, len(temp.index), 7)
ax.set_xticks(temp.index[x_indices])
_ = ax.set_xticklabels([d.strftime('%h %d (%a)') for d in temp.index[x_indices]], rotation=90)

### annotation
# annotate covid dates
dates = {
    '2020-03-20': 'Shelter In Place',
    '2020-06-08': 'Phase 1',
    '2020-06-22': 'Phase 2',
    '2020-07-06': 'Phase 3',
    '2020-07-20': 'Phase 4',
}
for date in dates: 
    ax.axvline(date, color='gold', linestyle='--')

# annotate storm dates
for date in ['2020-07-10', '2020-08-04']:
    ax.axvline(date, color='grey', linestyle='--')

# annotate heat advisory dates
for date in ['2020-07-19', '2020-07-23', '2020-07-26', '2020-08-10']:
    ax.axvline(date, color='salmon', linestyle='--')

## Goal 3: Understand social distancing complaints
### 4) Count a total number of social distancing complaints 

In [ ]:
query = {'$select': 'count(*)',
         '$where': 'descriptor in("Social Distancing", "Face Covering Violation")', 
        }
df = pd.read_csv(API+urlencode(query))
df.head()

In [ ]:
query = {'$select': 'min(created_date)',
         '$where': 'descriptor in("Social Distancing", "Face Covering Violation")', 
        }
df = pd.read_csv(API+urlencode(query))
df.head()

In [ ]:
query = {'$select': 'max(created_date)',
         '$where': 'descriptor in("Social Distancing", "Face Covering Violation")', 
        }
df = pd.read_csv(API+urlencode(query))
df.head()

### 5) Create a plot - daily social distancing complaint counts

In [ ]:
# Monthly counts of 311 complaints in 2021
query = {'$select': 'date_trunc_ymd(created_date) as date, count(*) as daily_counts',
         '$where': 'descriptor in("Social Distancing", "Face Covering Violation") and \
         created_date between "2020-03-28" and "2020-08-25"',
         '$group': 'date',
        }
df = pd.read_csv(API+urlencode(query), parse_dates=['date'])
df.index = df['date']
df.head()

In [ ]:
ax = df.daily_counts.plot.bar(figsize=(16,4))

# xlabels: print only once every week
xindices = list(range(0,len(df),7))
xlabels = df.index.strftime('%b %d (%a)')
ax.set_xticks(xindices)
ax.set_xticklabels(xlabels[xindices])
plt.xlabel('')

# Add annotations
dates = {
    '2020-03-20': 'Stay-at-home order',
    '2020-06-08': 'Phase 1',
    '2020-06-22': 'Phase 2',
    '2020-07-06': 'Phase 3',
    '2020-07-20': 'Phase 4',
}
annotation = [(i, date) for i, date in enumerate(df.index.strftime('%Y-%m-%d').tolist()) if date in dates]
for annot_i, annot_text in annotation: 
    ax.axvline(annot_i, color='gold', linestyle='--')
    ax.text(x=annot_i+1, y=2000, s='%s\n%s'%(dates[annot_text], 
                                               xlabels[annot_i]))

ax.grid(True, alpha=.3)
plt.ylabel('Number of SD Complaints')
min_date = df.index.min().strftime('%b %d')
max_date = df.index.max().strftime('%b %d')
_ = plt.title('Daily Social Distancing Complaint Volume (%s, 2020 - %s, 2020)'%(min_date, max_date))